In [4]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)
print(outputs)

tensor([0.6590, 0.2424, 0.0986])


In [6]:
# cross entropy loss in numpy

def cross_entropy(actual, predicted):
    loss = -np.sum(actual * np.log(predicted))
    return loss/float(predicted.shape[0])

# y must be one hot encoded
# if class 0: [1, 0, 0]
# if class 1: [0, 1, 0]
# if class 2: [0, 0, 1]

Y = np.array([1, 0, 0])


# y_pred has probabilities
Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)

print(f'Loss1 numpy: {l1:.4f}')
print(f'Loss2 numpy: {l2:.4f}')

Loss1 numpy: 0.1189
Loss2 numpy: 0.7675


In [7]:
# cross entropy loss in PyTorch
loss = nn.CrossEntropyLoss()

# Be careful that nn.CrossEntropyLoss applies nn.LogSoftmax + nn.NLLLoss
# No softmax in last layer

# Y (=labels) is not one-hot encoded
# Y_pred has raw scores (logits) (no softmax)

Y = torch.tensor([0])
# nsamples x nclasses = 1 x 3
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(l1.item())
print(l2.item())

0.4170299470424652
1.840616226196289


In [8]:
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)

print(predictions1)
print(predictions2)

tensor([0])
tensor([1])


In [10]:
# we have to predict a list of labels
Y = torch.tensor([2, 0, 1])
# note the number of samples is 3 now
# nsamples x nclasses = 3 x 3

Y_pred_good  = torch.tensor([[2, 0, 1], [0.4, 1, 2.3], [0.5, 2.0, 0.3]])
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3], [2, 0, 1], [0.4, 1, 2.3]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print('loss 1: ', l1.item())
print('loss 2: ', l2.item())

_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)

print(predictions1)
print(predictions2)


loss 1:  1.3334523439407349
loss 2:  1.3667856454849243
tensor([0, 2, 1])
tensor([1, 0, 2])


In [13]:
# multiclasses problem

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # no softmax at the end
        return out
    
model = NeuralNet(input_size=28*28, hidden_size=5, num_classes=3)
criterion = nn.CrossEntropyLoss() # (applies Softmax)